In [1]:
import shared_utils
import pandas as pd
from siuba import *

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


ModuleNotFoundError: No module named 'rt_analysis'

In [ ]:
shared_utils.gtfs_utils.get_trips?

In [7]:
bbb_trips = shared_utils.gtfs_utils.get_trips(300)


KeyboardInterrupt

